In [46]:
# import shutil
# shutil.rmtree("results")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split

competition_dataset_path = '/kaggle/input/llm-detect-ai-generated-text/train_essays.csv'
extra_dataset_path = '/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv'

competition_dataset = pd.read_csv(competition_dataset_path)
competition_dataset['label'] = competition_dataset['generated']
competition_dataset = competition_dataset.drop(columns=['prompt_id', 'id', 'generated'])

extra_dataset = pd.read_csv(extra_dataset_path)
extra_dataset = extra_dataset.drop(columns=['prompt_name', 'source', 'RDizzl3_seven'])

dataset = pd.concat([competition_dataset, extra_dataset], ignore_index=True)

condition = dataset['label'] == 0
filtered_dataset = dataset[condition]
random_rows = filtered_dataset.sample(n=11200)
dataset = dataset.drop(random_rows.index)

# condition = dataset['label'] == 1
# filtered_dataset = dataset[condition]
# random_rows = filtered_dataset.sample(n=16800)
# dataset = dataset.drop(random_rows.index)

df_train, df_test = train_test_split(dataset, test_size=0.30, random_state=42, shuffle=True, stratify=dataset["label"])

In [ ]:
dataset.label.value_counts()

In [ ]:
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

model_location = '/kaggle/input/bertdetectoraiessay/transformers/7500-0.0217/1'
# model_location = '/kaggle/input/bertdetectoraiessay/transformers/v1/1'
# model_location = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_location)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

train_dataset = Dataset.from_pandas(df_train)
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = Dataset.from_pandas(df_test)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

LABELS = ['human', 'generated']
id2label = {idx:label for idx, label in enumerate(LABELS)}
label2id = {label:idx for idx, label in enumerate(LABELS)}

model = AutoModelForSequenceClassification.from_pretrained(model_location, num_labels=len(LABELS), id2label=id2label, label2id=label2id)

training_args = TrainingArguments(
 output_dir="./results",
 num_train_epochs=3,  
 per_device_train_batch_size=8,  
 per_device_eval_batch_size=8,  
 evaluation_strategy="steps",  
 save_strategy="steps",  
 save_steps= 500,
 eval_steps = 500,
 logging_dir="./logs",  
 logging_steps=500,  
 learning_rate=5e-5,  
#  report_to='wandb'
 report_to='none'
)

trainer = Trainer(
 model=model,
 args=training_args,
 train_dataset=train_dataset,  
 eval_dataset=eval_dataset,
 tokenizer=tokenizer
)
# trainer.train()
# trainer.evaluate()

In [44]:
test_dataset_path = '/kaggle/input/llm-detect-ai-generated-text/test_essays.csv'

test_df = pd.read_csv(test_dataset_path)
test_df = test_df.drop(columns=['prompt_id'])

test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(preprocess_function, batched=True)

predictions, label_ids, metrics = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions, axis=1)

out =  pd.DataFrame(data={'id': test_df.id, 'generated': predicted_labels})
# out.to_csv('submission.csv', index=False)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]